<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-4/blob/master/NLP_capstone_Data_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw

    100% |████████████████████████████████| 122kB 6.1MB/s 
    100% |████████████████████████████████| 204kB 8.7MB/s 


In [0]:
import requests
import praw
import time
import pandas as pd
import json
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore')

In [0]:

reddit = praw.Reddit(user_agent='Scrape_bot',
                     client_id='_XX9OWL3X9Gv-A', client_secret="vAvTdaG9JYkrjd2Pe2_r5Q3wrIo")

In [0]:
subreddit = reddit.subreddit('Politics')
top_subreddit = subreddit.top(limit=50)


In [22]:
for submission in subreddit.top(limit=1):
    print(submission.title, submission.id)

Kim Davis, clerk who refused to sign marriage licenses for gay couples, loses to Democrat 9uuhl3


In [0]:
submissions_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[], 
                "comms_num": [], 
                "created": []} 
                #"body":[]}

for submission in top_subreddit:
    submissions_dict["title"].append(submission.title)
    submissions_dict["score"].append(submission.score)
    submissions_dict["id"].append(submission.id)
    submissions_dict["url"].append(submission.url)
    submissions_dict["comms_num"].append(submission.num_comments)
    submissions_dict["created"].append(submission.created)
    #topics_dict["body"].append(submission.selftext) # ignore self text posts

In [0]:
submissions_data = pd.DataFrame(submissions_dict)

In [25]:
submissions_data.head()

,comms_num,created,id,score,title,url
0,2771,1.541553e+09,9uuhl3,101764,"Kim Davis, clerk who refused to sign marriage ...",https://www.kentucky.com/news/politics-governm...
1,20873,1.494882e+09,6bd42j,99354,Trump revealed highly classified information t...,https://www.washingtonpost.com/world/national-...
2,14557,1.516929e+09,7t12e6,95202,"Trump Ordered Mueller Fired, but Backed Off Wh...",https://www.nytimes.com/2018/01/25/us/politics...
3,6544,1.542671e+09,9ymbzc,89715,Ivanka Trump used a personal email account to ...,https://www.washingtonpost.com/politics/ivanka...
4,1947,1.511730e+09,7fpqwf,88282,A petition calling for FCC Chairman Ajit Pai t...,https://www.dailydot.com/layer8/we-the-petitio...


In [0]:
submission = [reddit.submission(x) for x in submissions_data['id']]

In [0]:
# This function takes a given reddit submission id and does the following:
    # cleans comment threads of instances of "More Comments"
    # gets a list of reddit comment tree ids
    # Grabs the body of every Root comment
top_comments = []
for x in submission[0:50]:
  (x).comments.replace_more(limit=0)
  comments = list(x.comments)
  root_comments = [y.body for y in comments]
  top_comments.extend(root_comments)  
    

In [28]:
len(top_comments)

5472

In [29]:
# lets start cleaning comments


top_comments[0:10]

['\nAs a reminder, this subreddit [is for civil discussion.](/r/politics/wiki/index#wiki_be_civil)\n\nIn general, be courteous to others. Attack ideas, not users. Personal insults, shill or troll accusations, hate speech, **any** advocating or wishing death/physical harm, and other rule violations can result in a permanent ban. \n\nIf you see comments in violation of our rules, please report them.\n\n***\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/politics) if you have any questions or concerns.*',
 'Good job Kentucky! ',
 'title should be: "Lady who refused to do job, loses job."',
 'From what I remember, this awful bitch owes the people of Kentucky some goddamn money... And human beings, in general, an apology',
 "Guess it was God's will ¯\\\\\\_(ツ)\\_/¯",
 'Good, now she can go back to husband number, what was it, four?',
 '700 votes. Remember every vote matters.',
 'It appears that an all p

# Clean the text

https://medium.com/ml2vec/using-word2vec-to-analyze-reddit-comments-28945d8cee57

https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72

In [60]:

import nltk.data;

import spacy
nlp = spacy.load('en', parse=True, tag=True, entity=True)
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

from gensim.models import word2vec;

from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;


import os
import re

import logging
import time
import sys
import multiprocessing

from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from itertools import cycle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle');

## Expanding Contractions

In [0]:

#Contraction Map is located in at the bottom of the book and is a shameless copy of contraction mappings
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [0]:
df_politics['comments'] = df_politics['comments'].apply(lambda x:expand_contractions(x))

In [0]:
df_politics['comments']

## Removing Special characters

In [0]:
# BIG THANKS 
cleaned_comments = []
for pos in top_comments:

    #Get the comment
    
    #Normalize tabs and remove newlines
    no_tabs = str(pos).replace('\t', ' ').replace('\n', '');

    #Remove all characters except A-Z and a dot.
    alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs);

    #Normalize spaces to 1
    multi_spaces = re.sub(" +", " ", alphas_only);

    #Strip trailing and leading spaces
    no_spaces = multi_spaces.strip();

    #Normalize all charachters to lowercase
    clean_text = no_spaces.lower();

    #Get sentences from the tokenizer, remove the dot in each.
    sentences = tokenizer.tokenize(clean_text);
    sentences = [re.sub("[\.]", "", sentence) for sentence in sentences];
    cleaned_comments.extend(sentences)
    
    #If the text has more than one space (removing single word comments) and one character, write it to the file.
    #if len(clean_text) > 0 and clean_text.count(' ') > 0:
    #    for sentence in sentences:
    #        out_file.write("%s\n" % sentence)
    #        print(sentence);
    


In [34]:
len(cleaned_comments)

12507

In [36]:
cleaned_comments[0:10]

['as a reminder this subreddit is for civil discussion',
 'r politics wiki index wiki be civil in general be courteous to others',
 'attack ideas not users',
 'personal insults shill or troll accusations hate speech any advocating or wishing death physical harm and other rule violations can result in a permanent ban',
 'if you see comments in violation of our rules please report them',
 'i am a bot and this action was performed automatically',
 'please contact the moderators of this subreddit message compose to r politics if you have any questions or concerns',
 'good job kentucky',
 'title should be lady who refused to do job loses job',
 'from what i remember this awful bitch owes the people of kentucky some goddamn money and human beings in general an apology']

## Lemmatize the text

In [0]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [0]:
df_politics = pd.DataFrame(cleaned_comments,columns=['comments'])
df_politics['subreddit'] = 'politics'

In [51]:
df_politics.head()

,comments,subreddit
0,as a reminder this subreddit is for civil disc...,politics
1,r politics wiki index wiki be civil in general...,politics
2,attack ideas not users,politics
3,personal insults shill or troll accusations ha...,politics
4,if you see comments in violation of our rules ...,politics


In [0]:
df_politics['comments']= df_politics['comments'].apply(lambda x : lemmatize_text(x))


## Remove Stop Words

In [0]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


In [0]:
df_politics['comments'] = df_politics['comments'].apply(lambda x:remove_stopwords(x))

In [74]:
df_politics['comments'].head()

0    as a reminder this subreddit be for civil disc...
1    r politic wiki index wiki be civil in general ...
2                                 attack idea not user
3    personal insult shill or troll accusation hate...
4    if you see comment in violation of our rule pl...
Name: comments, dtype: object

## TFIDF Vectorizer

# Word 2 Vec

# Other Code

In [0]:
bb_url = 'https://www.reddit.com/r/Politics/.json'
bb_headers = {'User-agent': 'Scrape_bot'}

In [0]:
bb_res = requests.get(bb_url, headers=bb_headers)

In [0]:
bb_res.status_code

200

In [0]:
bb_json = bb_res.json()

In [0]:
bb_json

In [0]:
sorted(bb_json.keys())

['data', 'kind']

In [0]:
sorted(bb_json['data'].keys())

['after', 'before', 'children', 'dist', 'modhash']

In [0]:
bb_json['data']['children'][0]['data']

In [0]:
bb_posts = []
bb_after = None
#bb_after_list = []

for bbp in range(48):
    #print(bbp)
    if bb_after == None:
        bb_params = {}
    else:
        bb_params = {'after': bb_after}
        
    bb_res = requests.get(bb_url, params=bb_params, headers=bb_headers)
    if bb_res.status_code == 200:
        bb_json = bb_res.json()
        bb_posts.extend(bb_json['data']['children'])
        bb_after = bb_json['data']['after']
        #bb_after_list.append(bb_after)

    else:
        print('Status Code Error', bb_res.status_code)
        break
    time.sleep(.01)

In [0]:
bb_posts 

In [0]:
len(submissions)

1195

In [0]:
print(reddit.read_only)

True


In [0]:
submissions = [reddit.submission(x['data']['id']) for x in bb_posts]

In [0]:
 submissions

In [0]:
top_comments = pd.Dataframe([])
for top_level_comment in submissions[1].comments:
    (top_level_comment.body)

In [0]:
submissions[1].comments

In [0]:

parent_comments = []

for links in submissions[0:10]:
  links.comments.replace_more(limit=0)
  for top_level_comments in links.comments:
      parent_comments.extend(top_level_comment.body)

In [0]:
submissions[2].comments.replace_more(limit=0)

[]

In [0]:
submissions[5].comments.body

AttributeError: ignored

In [0]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}